In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    "../data/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
    subset="training"
)
val_generator = train_datagen.flow_from_directory(
    "../data/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.


In [3]:
inputs = tf.keras.Input(shape=(150, 150, 3), name="image")

# Use the same input tensor for ResNet50 so it doesn't create another InputLayer
base = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=inputs
)

# Freeze base model
base.trainable = False

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="catsdogs_resnet50")

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(train_generator, validation_data=val_generator, epochs=5)

c:\Users\Mana\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 102s 494ms/step - accuracy: 0.5359 - loss: 0.6919 - val_accuracy: 0.5965 - val_loss: 0.6792
Epoch 2/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 57s 282ms/step - accuracy: 0.5643 - loss: 0.6780 - val_accuracy: 0.6252 - val_loss: 0.6702
Epoch 3/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 58s 288ms/step - accuracy: 0.5657 - loss: 0.6757 - val_accuracy: 0.6259 - val_loss: 0.6709
Epoch 4/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 58s 290ms/step - accuracy: 0.5792 - loss: 0.6727 - val_accuracy: 0.5740 - val_loss: 0.6734
Epoch 5/5
201/201 ━━━━━━━━━━━━━━━━━━━━ 58s 291ms/step - accuracy: 0.5865 - loss: 0.6706 - val_accuracy: 0.6327 - val_loss: 0.6546


In [5]:
acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
print(f"Train Accuracy: {acc:.2f}, Validation Accuracy: {val_acc:.2f}")

model.save("../models/cats_dogs_resnet.h5")

Train Accuracy: 0.59, Validation Accuracy: 0.63
